In [25]:
import clr
from bonsai_pylib import load_bonsai_config
load_bonsai_config()

# Add reference to the Harp package
clr.AddReference("Harp.VestibularH1")
import Harp.VestibularH1 as h1Dev
clr.AddReference("Bonsai.Harp")
import Bonsai.Harp as harp


In [26]:
## Connect to the device usign the Async Device class
dev = h1Dev.VestibularH1Device("COM5")
# Call the Dispose method to close the connection
#dev.Dispose()

In [27]:
# Get the map of available registers
RegisterMap = h1Dev.Device().RegisterMap
AvailableKeys = list(h1Dev.Device().RegisterMap.Keys)

# Get the Name of the register:
print(RegisterMap[AvailableKeys[0]])

Cam0Event


In [28]:
for key in AvailableKeys:
    method = f"Read{RegisterMap[key]}Async"
    try:
        result = getattr(dev, method)().Result
        print(result)
    except:
        pass

0
0
100
0
0
68
0
Harp.VestibularH1.InStatePayload
0
0
0
7


In [29]:
# Make an asynchronous call to read a register from the device
Cam0Freq = dev.ReadCam0TriggerFrequencyAsync().Result
print([type(Cam0Freq), Cam0Freq])

# Some registers returned complex parsed values that can be accessed through properties
InState = dev.ReadInStateAsync().Result
print([type(InState), InState])
print(InState.Lick0)
print(InState.Lick0Changed)
#...

[<class 'int'>, 100]
[<class 'Harp.VestibularH1.InStatePayload'>, <Harp.VestibularH1.InStatePayload object at 0x000001BCD808ADC0>]
False
False


In [30]:
# Return an harp message from an async read
# Note, the correct method still needs to be applied (i.e. "ReadByte")
harpMessage = dev.CommandAsync(harp.HarpCommand.ReadByte(h1Dev.InState.Address)).Result
print(harpMessage)

Read 39 TimestampedU8@180.0145 Length:1


In [31]:
## Write to a register
value = 20
Cam0Freq = dev.WriteCam0TriggerFrequencyAsync(value).Result
# Confirm the value of the register has changed by reading it back
Cam0Freq = dev.ReadCam0TriggerFrequencyAsync().Result
print(f"Write value: {value}, Read value: {Cam0Freq}")

Write value: 20, Read value: 20


In [32]:
# Write an harp message directly
# Note, the correct method still needs to be applied (i.e. "WriteUInt16")
harpMessage = dev.CommandAsync(
    harp.HarpCommand.WriteUInt16(
        h1Dev.Cam0TriggerFrequency.Address, 100)).Result
print(harpMessage)

Write 34 TimestampedU16@180.22048 Length:1


In [33]:
# Manipulate HarpMessage
an_harp_message = harp.HarpMessage.FromSingle(32, harp.MessageType.Write, 1.0)
print(harp.HarpMessage.get_Address(an_harp_message))
print(harp.HarpMessage.get_PayloadType(an_harp_message))
print(harp.HarpMessage.GetPayloadSingle(an_harp_message))

32
Float
1.0


In [34]:
dev.Dispose() # Make sure to dispose the device when done
